In [3]:
# Add new CNB data

import os
import glob
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import re
import shutil

In [22]:
# Set paths
fcpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/fmri/postprocessed/3T/xcp_d"
outpath = "~/Desktop/ImageData/PMACS_remote/analysis/postprocessing/"
clinpath = "~/Desktop/ImageData/PMACS_remote/data/clinical"
cestpath = "~/Desktop/ImageData/PMACS_remote/data/cest"

# Choose what to analyse
networks = ["Cont", "Default", "DorsAttn", "Vis", "SalVentAttn", "SomMot", "Limbic"] 
btwn_ntwks = ["Cont_btwn", "Default_btwn", "DorsAttn_btwn", "Vis_btwn", "SalVentAttn_btwn", "SomMot_btwn", "Limbic_btwn"] 
all_ntwks = ["Cont_all", "Default_all", "DorsAttn_all", "Vis_all", "SalVentAttn_all", "SomMot_all", "Limbic_all"] 

CESTnetworks = ["avgCEST_Cont", "ctCEST_Cont", "avgCEST_Default", "ctCEST_Default","avgCEST_SomMot", "ctCEST_SomMot", "avgCEST_DorsAttn", "ctCEST_DorsAttn", "avgCEST_Vis", "ctCEST_Vis","avgCEST_Limbic", "ctCEST_Limbic",  "avgCEST_SalVentAttn","ctCEST_SalVentAttn"]
CNB_scores = ["tap_tot"]
CNB_valids = ["tap_valid"] 
diag_scores = ["hstatus","age"]
demo_scores = ["sex", "race","ethnic","dateDiff"]
diag_details = ["axis1_desc1", "axis1_desc2", "axis1_desc3","axis1_desc4","axis1_desc5", "axis1_desc6"]


# Make dataframe based on metrics of interest
grp_df = pd.DataFrame(columns = ["BBLID"] + ["Session"] + ["SCANID_CEST"] + demo_scores + networks + btwn_ntwks + CESTnetworks + CNB_scores + diag_scores)
diag_df = pd.DataFrame(columns = ["BBLID"] + ["Session"] + ["SCANID_CEST"] + diag_scores + diag_details)
print(grp_df)

# Initialize empty lists and vars
bblids = []
sesids = []
gluseses = []

cestmat = pd.read_csv(cestpath + "/compiled_outputs/compiled_cortical_UNI.csv", sep=',')
# cestmat = pd.read_csv(clinpath + "/demographics.csv", sep='\t') add grp CEST map here
cestmat.set_index('Subject', inplace = True)

Empty DataFrame
Columns: [BBLID, Session, SCANID_CEST, sex, race, ethnic, dateDiff, Cont, Default, DorsAttn, Vis, SalVentAttn, SomMot, Limbic, Cont_btwn, Default_btwn, DorsAttn_btwn, Vis_btwn, SalVentAttn_btwn, SomMot_btwn, Limbic_btwn, avgCEST_Cont, ctCEST_Cont, avgCEST_Default, ctCEST_Default, avgCEST_SomMot, ctCEST_SomMot, avgCEST_DorsAttn, ctCEST_DorsAttn, avgCEST_Vis, ctCEST_Vis, avgCEST_Limbic, ctCEST_Limbic, avgCEST_SalVentAttn, ctCEST_SalVentAttn, tap_tot, hstatus, age]
Index: []

[0 rows x 38 columns]


In [25]:
# Read in existing grp_df and new cnb
cnbmat = pd.read_csv("/Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/motor_glu_pipeline/maggie_datarequest_fullcnb1_0521_edited.csv", sep=',') 
grp_df = pd.read_csv("/Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/motor_glu_pipeline/grp_df_3T_allCNB.csv", sep=',') 
scan_dates = pd.read_csv("/Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/motor_glu_pipeline/scan_dates.csv", sep=',') 

CNB_valids = ["tap_valid"]
CNB_scores = ["tap_tot"]

cnbmat.set_index('bblid', inplace = True)
scan_dates.set_index('BBLID', inplace = True)
#grp_df.insert(48,'lagtime_CNB', 'NaN')
#grp_df.insert(49,'lagtime_scan', 'NaN')
#print(grp_df)

In [26]:
for bblid in grp_df['BBLID'].astype(int):
    for i in range(len(CNB_scores)):
        CNB_score = CNB_scores[i]
        CNB_valid = CNB_valids[i]
        scores = cnbmat[CNB_score]
        if int(bblid) in scores.index:
            score = scores[int(bblid)]
            valids = cnbmat[CNB_valid]
            valid = str(valids[int(bblid)])
            lagtimes = cnbmat['lagtime']
            lagtime = lagtimes[int(bblid)]
            if 'V' in valid: 
                grp_df.loc[grp_df['BBLID'] == bblid, grp_df.columns == CNB_score] = score 
                grp_df.loc[grp_df['BBLID'] == bblid, grp_df.columns == 'lagtime_CNB'] = lagtime
    if int(bblid) in scan_dates.index:
        scanlags = scan_dates['Lagtimes']
        scanlag = scanlags[int(bblid)]
        grp_df.loc[grp_df['BBLID'] == bblid, grp_df.columns == 'lagtime_scan'] = scanlag


In [36]:
runcest = True

##################################################################################################
## CEST
##################################################################################################
for bblid in grp_df['BBLID'].astype(int):
    if runcest and bblid != "88760": 
        #print("Processing " + bblid + "'s CEST data'")
        # Extract Glu Session ID
        gluses = grp_df.loc[grp_df['BBLID'].astype(int) == bblid, grp_df.columns == 'SCANID_CEST'].values[0].astype(str) 
        cestid = str(bblid) + "_" + str(gluses[0])
        print(cestid)
        # Import data
        for network in networks:
            col_name = "avgCEST_" + network # for grp_df
            ct_name = "ctCEST_" + network # for grp_df
            col_cestmat = network + " NZMean"
            ct_cestmat = network + " NZcount"
            if str(cestid) in cestmat.index:    
                #print(cest_avg)
                cest_cts = cestmat[ct_cestmat]
                cest_ct = cest_cts[str(cestid)]
                print(ct_name)
                print(cest_ct)
                grp_df.loc[grp_df['BBLID'].astype(int) == bblid, grp_df.columns == ct_name] = cest_ct                            

print(grp_df)

116354_11774
ctCEST_Cont
591
ctCEST_Default
782
ctCEST_DorsAttn
278
ctCEST_Vis
270
ctCEST_SalVentAttn
1288
ctCEST_SomMot
449
ctCEST_Limbic
167
96465_12069
ctCEST_Cont
1275
ctCEST_Default
1480
ctCEST_DorsAttn
201
ctCEST_Vis
141
ctCEST_SalVentAttn
1551
ctCEST_SomMot
649
ctCEST_Limbic
233
126532_12582
ctCEST_Cont
1461
ctCEST_Default
1415
ctCEST_DorsAttn
244
ctCEST_Vis
282
ctCEST_SalVentAttn
1078
ctCEST_SomMot
499
ctCEST_Limbic
240
92211_10981
ctCEST_Cont
1154
ctCEST_Default
803
ctCEST_DorsAttn
222
ctCEST_Vis
227
ctCEST_SalVentAttn
1446
ctCEST_SomMot
633
ctCEST_Limbic
23
132179_10760
ctCEST_Cont
893
ctCEST_Default
677
ctCEST_DorsAttn
104
ctCEST_Vis
129
ctCEST_SalVentAttn
1007
ctCEST_SomMot
486
ctCEST_Limbic
217
135277_12808
ctCEST_Cont
1074
ctCEST_Default
1306
ctCEST_DorsAttn
201
ctCEST_Vis
250
ctCEST_SalVentAttn
1622
ctCEST_SomMot
515
ctCEST_Limbic
254
98370_12558
ctCEST_Cont
1497
ctCEST_Default
1746
ctCEST_DorsAttn
226
ctCEST_Vis
90
ctCEST_SalVentAttn
1650
ctCEST_SomMot
572
ctCEST_Limbic

In [37]:
print(grp_df)
grp_df.to_csv('grp_df_3T_allCNB_with_counts.csv', index=True)

    Unnamed: 0   X   BBLID Session  SCANID_CEST sex  race  ethnic  dateDiff  \
0            0  27  116354   11647        11774   F   2.0     2.0      53.0   
1            1  81   96465   11557        12069   M   1.0     2.0     117.0   
2            2  51  126532   12260        12582   F   1.0     2.0      11.0   
3            3  75   92211   10435        10981   F   2.0     2.0       9.0   
4            4   7  132179   10780        10760   F   2.0     2.0       0.0   
..         ...  ..     ...     ...          ...  ..   ...     ...       ...   
67          67  34   93274   10785        10765   F   5.0     2.0      11.0   
68          68  62  128079   11639        11934   M   2.0     2.0      57.0   
69          69   2  128865   12165        12325   M   1.0     2.0      49.0   
70          70  14   96902   11870        11903   F   1.0     1.0       6.0   
71          71  61   85743   10974        10944   F   1.0     2.0       2.0   

        Cont  ...  SomMot_all  Limbic_all    cest_f